# DMS-MaPseq

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from util import *
from config import *
import plots
import ipynbname

import plotly.graph_objects as go
from plotly.subplots import make_subplots

### # Mutations per read
- histogram
- x-axis = # mutations
- y-axis = # reads


In [ ]:
unique_samples = study.df['sample'].unique()
fig = make_subplots(rows=len(unique_samples), cols=1, vertical_spacing=0.2/len(unique_samples),
                     subplot_titles=['Number of mutations per read - {}'.format(sample) for sample in unique_samples])
for i_s, sample in enumerate(unique_samples):
    fig.add_trace( plots.mutations_per_read(study, sample), row=i_s+1, col=1 )
    fig.update_yaxes(title='Count')
    fig.update_xaxes(dtick=10)


fig.update_layout(autosize=True, height=5000, title='Number of mutation per read across samples')

save_plotly_fig(ipynbname.path(), '[A] Mutations per read', fig)
# fig.show()

In [ ]:
fig = make_subplots(cols=1, rows=2)

x=[1,2,3]
y1=[5,2,7]
y2=[0,9,4]

fig.add_trace( go.Bar(x=x, y=y1, showlegend=False), col=1, row=1 )
fig.add_trace( go.Bar(x=x, y=y2, showlegend=False), col=1, row=1 )

fig.add_trace( go.Bar(x=x, y=y1, showlegend=False), col=1, row=2 )
fig.add_trace( go.Bar(x=x, y=y1, showlegend=False), col=1, row=2 )

for trace, name in zip(fig["data"][:2], ['A', 'B']):
     trace.update(showlegend=True)
     trace["name"] = name

fig.update_layout(barmode='stack')

# fig.update_layout(legend=dict(x=0, y=1, traceindex = [0,1], font=dict(family="sans-serif", size=12, color="black"), bgcolor="white", bordercolor="black", borderwidth=2))

### Mutation identity at each position
- stacked bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [ ]:
%autoreload 2
for sample in study.df['sample'].unique():
    
    unique_constructs = study.get_df(sample=sample)['construct'].unique()
    seg_len = 100
    for i_seg in range(0, len(unique_constructs), seg_len):

        sub_constructs = unique_constructs[i_seg:i_seg+seg_len]
        print(i_seg, i_seg+seg_len)

        fig = make_subplots(rows=len(sub_constructs), cols=1, vertical_spacing=0.2/len(sub_constructs),
                            subplot_titles=['Mutation identity at each position - {} - {}'.format(sample, cst) for cst in sub_constructs])
        for i_c, construct in enumerate(sub_constructs):
            muts_identity = plots.mutation_identity_at_each_position(study, sample, construct)

            for bar in muts_identity['fig']:
                fig.add_trace( bar, row=i_c+1, col=1 )
                
            # savefig2(ipynbname.path(), '[B] Mutation identity at each position/{}/{}'.format(sample, construct))
        for trace, name in zip(fig["data"][:4], ['A','C','G','T']):
            trace.update(showlegend=True)
            trace["name"] = name
        
        fig.update_xaxes(tickangle=0, tickvals=np.arange(len(muts_identity['data'].index)), ticktext=list(muts_identity['data'].index), tickfont={'size':8})
        fig.update_yaxes(title='Mutation fraction')
        fig.update_layout(barmode='stack', height=500*len(sub_constructs), width=1500)
        save_plotly_fig(ipynbname.path(), '[B] Mutation identity at each position/{}/{}'.format(sample, i_seg), fig)
        fig.show()
        break
    break

### Mutation fraction at each position 
- bar graph (ACUG coloring)
- x-axis = position (number/base/both??)
- y-axis = # reads


In [ ]:
for sample in study.df['sample'].unique():
    for construct in study.get_df(sample=sample)['construct'].unique():
        plots.mutation_fraction_at_each_position(study, sample, construct)
        savefig2(ipynbname.path(), '[C] Mutation fraction at each position/{}/{}'.format(sample, construct))
        if not generate_plots:
            break  
    if not generate_plots:
        break  

### Read coverage per position
- bar graph
- x-axis = position (number/base/both??)
- y-axis = coverage fraction

In [ ]:

for sample in study.df['sample'].unique():
    for construct in study.get_df(sample=sample)['construct'].unique():
        try:
            plots.read_coverage_per_position(study, sample, construct)
            savefig2(ipynbname.path(), '[D] Read coverage per position/{}/{}'.format(sample, construct))
        except:
            print('Error in sample {} construct {}'.format(sample, construct))
        if not generate_plots:
            break  
    if not generate_plots:
        break  